**NOTE**: to get this working, you must install the `data-analysis` group dependencies using Poetry:

```shell
poetry install --with=data-analysis
```

You should also run this if you are using git:

```shell
poetry run nbstripout --install
```

This will automatically clear output when committing to git :)

In [ ]:
import json
import pickle
import sqlite3
from operator import itemgetter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import cohen_kappa_score

from data_analysis_utils import agreement_as_label

Ways to index and group data:

In [ ]:
# by each Java source code file (context or unit):
BY_CONTEXT = ["srcml_path", "version"]
# by the PEM variant shown to the rater:
BY_SCENARIO = BY_CONTEXT + ["variant"]
# by the rater: this uniquely identifies one particular data point:
BY_RATER = BY_SCENARIO + ["rater"]

We might as well load all of the scenarios, indexed by `srcml_path` and `version`:

In [ ]:
with open("sample.pickle", "rb") as sample_file:
    _ALL_SCENARIOS = pickle.load(sample_file)

ALL_SCENARIOS = {
    (scenario["xml_filename"], scenario["version"]): scenario
    for scenario in _ALL_SCENARIOS
}

I will also load the Decaf PEMs, because it will be necessary later to manually compare Decaf's output to the javac PEM under question.

In [ ]:
with open("decaf.pickle", "rb") as decaf_file:
    DECAF_RESPONSES = pickle.load(decaf_file)

# Loading the answers

We'll create a basic dataframe from `answers.sqlite3`. However, the `answers` column will need to be parsed as JSON.

In [ ]:
# Read answers.sqlite3 into a DataFrame
conn = sqlite3.connect("answers.sqlite3")
df = pd.read_sql_query("SELECT * FROM answers", conn)
conn.close()

# Extracting the data from the JSON column

Now time to clean the data — extract it from that JSON column.

Instead of creating a nice schema in `answers.sqlite3`, I decided to defer the job of making nice columns to the data analysis stage. So now we have to parse the answers column as JSON and extract data.


As of 2023-05-02, these are columns:

In [ ]:
json.loads(df["answers"][0])

In [ ]:
def json_to_columns(df):
    json_column = df["answers"].apply(json.loads)
    return df.assign(
        jargon=(
            json_column.apply(itemgetter("jargon")).astype(int)
        ),
        sentence_structure=(
            json_column.apply(itemgetter("sentence_structure")).astype("category")
        ),
        # Explanation should already be a boolean column, that is completely filled in:
        explanation=(
            json_column.apply(itemgetter("explanation"))
        ),
        explanation_correctness=(
            json_column.apply(itemgetter("explanation_correctness")).astype("category")
        ),
        # I wish I had a better name for this column, but it's basically, "if the explanation is MAYBE correct, WHY is it maybe correct?"
        explanation_maybe=(
            json_column.apply(itemgetter("explanation_maybe")).astype("category")
        ),
        fix=(
            json_column.apply(itemgetter("fix")).fillna("N/A").astype("category")
        ),
        fix_correctness=(
            json_column.apply(itemgetter("fix_correctness")).astype("category")
        ),
        additional_errors=(
            json_column.apply(itemgetter("additional_errors")).astype("category")
        ),
        notes=(
            json_column.apply(itemgetter("notes")).astype("string")
        ),
        length=(
            json_column.apply(itemgetter("length")).astype(int)
        ),
    )

def variant_as_categorical(df):
    variant = df["variant"].astype("category")
    assert len(variant.cat.categories) == 4
    return df


def rater_as_categorical(df):
    rater = df["rater"].astype("category")
    assert len(rater.cat.categories) == 3
    return df.assign(rater=rater)


def set_empty_notes_to_na(df):
    "Notes that are empty strings should just be missing values"
    return df.assign(notes=df["notes"].replace("", pd.NA))

expanded_df = df\
    .pipe(json_to_columns)\
    .drop(columns=["answers"])\
    .pipe(variant_as_categorical)\
    .pipe(rater_as_categorical)\
    .pipe(set_empty_notes_to_na)

expanded_df.sample(5)

In [ ]:
expanded_df.info()

What are the levels of the categorical variables?

In [ ]:
# Get the levels for each categorical variable in df
def get_levels(df, skip=()):
    for column in df.select_dtypes("category").columns:
        if column in skip:
            continue
        print(df[column].value_counts())
        print(set(df[column].cat.categories))
        print()


get_levels(expanded_df, skip=["variant", "rater"])

# Creating `full_df`

For further data analysis, it's useful to think of some of the categorical responses as being ordinal. It will also smooth over inter-rater reliability (you could argue that it's cooking the books), since if two raters answer "yes" and "maybe", that's more agreement than two raters saying "yes" and "no".

There are also some columns that can be made binary, like "does it provide a correct fix?" or "is the explanation definitely correct?"

In [ ]:
REMAPPINGS = dict(
    sentence_structure_ordinal={"unclear": 0, "could-be-clearer": 1, "clear": 2},
    explanation_correctness_ordinal={"no": 0, "maybe": 1, "yes": 2},
    fix_ordinal={
        "no": 0,
        "implicit-suggestion": 1,
        "generic": 2,
        "hint": 3,
        "confident": 4,
    },
    fix_correctness_ordinal={"no": 0, "maybe": 1, "yes": 2},
    additional_errors_ordinal={"no": 0, "maybe": 1, "yes": 2},
    # Is the sentence structure clear?
    sentence_structure_binary={"unclear": 0, "could-be-clearer": 0, "clear": 1},
    # Can the explanation be reasonably considered to be correct?
    explanation_correctness_binary={"no": 0, "maybe": 1, "yes": 1},
    # Is **any** kind of fix suggested?
    fix_binary_levels={
        "no": 0,
        "implicit-suggestion": 1,
        "generic": 1,
        "hint": 1,
        "confident": 1,
    },
    # Is the fix DEFINITELY correct?
    fix_correctness_binary={"no": 0, "maybe": 0, "yes": 1},
    additional_errors_binary={"no": 0, "maybe": 1, "yes": 1},
)

def with_correct_dtype(df, column_name):
    return df[column_name].map(REMAPPINGS[column_name])


def add_ordinal_columns(df):
    return df.assign(
        sentence_structure_ordinal=df["sentence_structure"].map(REMAPPINGS["sentence_structure_ordinal"]),
        explanation_correctness_ordinal=df["explanation_correctness"].map(REMAPPINGS["explanation_correctness_ordinal"]),
        fix_ordinal=df["fix"].map(REMAPPINGS["fix_ordinal"]),
        fix_correctness_ordinal=df["fix_correctness"].map(REMAPPINGS["fix_correctness_ordinal"]),
        additional_errors_ordinal=df["additional_errors"].map(REMAPPINGS["additional_errors_ordinal"]),
    )


def add_binary_columns(df):
    return df.assign(
        sentence_structure_binary=df["sentence_structure"].map(REMAPPINGS["sentence_structure_binary"]),
        explanation_correctness_binary=df["explanation_correctness"].map(REMAPPINGS["explanation_correctness_binary"]),
        fix_binary=df["fix"].map(REMAPPINGS["fix_binary_levels"]),
        fix_correctness_binary=df["fix_correctness"].map(REMAPPINGS["fix_correctness_binary"]),
        additional_errors_binary=df["additional_errors"].map(REMAPPINGS["additional_errors_binary"]),
    )


## The perfect fix

A perfect rating is when an error message:
 1. has an explanation
 2. ...that is correct
 3. with a confident fix
 3. ...that is correct.

In [ ]:
def add_perfect_fix(df):
    return df.assign(
        perfect_fix=(
            # df["explanation_correctness"] being not null occurs if and only if df["explanation"] == True
            (df["explanation_correctness"] == "yes") &
            (df["fix"] == "confident") &
            (df["fix_correctness"] == "yes")
        ).astype(bool)
    )

In [ ]:
full_df = expanded_df\
    .pipe(add_ordinal_columns)\
    .pipe(add_binary_columns)\
    .pipe(add_perfect_fix)

full_df.sample(5)

Every rating **must** state whether an explanation was provided or not. Assert this here:

In [ ]:
assert full_df["explanation"].isna().sum() == 0

## Creating `lengths`

The length should be the equal regardless of the rater, so let's assert that here and get a table we can join with later.

In [ ]:
def assert_lengths(df):
    "Ensure that each rater has seen a message with the same length."
    assert (df["length"]["min"] == df["length"]["max"]).all()
    return df


def as_sql_style_table(df):
    return pd.DataFrame({
        "srcml_path": df["srcml_path"],
        "version": df["version"],
        "variant": df["variant"],
        "length": df["length"]["min"],
    })


# A common pitfall of using a categorical variable with groupby
# is that it does "some sort of cartesian product" with the other columns,
# which is not what we want.
# So we have to set `observed=False` to avoid this.
# 
# See: https://stackoverflow.com/a/67645084
message_lengths = full_df\
    .groupby(BY_SCENARIO, as_index=False, observed=False)\
    .agg({"length": ["min", "max"]})\
    .pipe(assert_lengths)\
    .pipe(as_sql_style_table)

message_lengths.sample(5)

# Data cleaning

There are a few rows we should remove before doing further analysis. These rows come from **Decaf**. Unfortunately, some of these Decaf rows correspond to a _different_ Java error than the one under study, so it would be unfair/invalid to include Decaf in these analyses.

## Finding and removing ratings that should be skipped

When I was rating, I marked a few Decaf PEMs as skippable, because Decaf did not match the same error message as the other variants. In other words, **Decaf was explaining the wrong error**. It's unfair to include these PEMs in the analysis, so let's find them and skip them:

In [ ]:
def select_marked_as_skip(df):
    return df[df["notes"] == "SKIP"]

def ensure_variant_is_decaf(df):
    assert (df["variant"] == "decaf").all()
    return df

def to_scenario_keys(df):
    """
    Converts the data frame into a set of keys that can be used 
    to look up the code context in ALL_SCENARIOS.
    """
    return set(df[BY_CONTEXT].itertuples(index=False, name=None))


should_skip = full_df\
    .pipe(select_marked_as_skip)\
    .pipe(ensure_variant_is_decaf)\
    .pipe(to_scenario_keys)

should_skip

Prajish found a few more Decaf PEMs that should be skipped.

I... also might not have been diligent when writing SKIP in the column...

In [ ]:
suspected_filenames = {"Quiz.java", "getLargestPerimeterMultipleFiles.java", "Game.java", "PerimeterAssignmentRunner.java"}

suspected_contexts = []
for (srcml_path, version), scenario in ALL_SCENARIOS.items():
    unit = scenario["unit"]
    if unit.filename in suspected_filenames:
        suspected_contexts.append((srcml_path, version))

suspected_contexts

Okay, let's manually inspect these. I'm looking for errors that will be caugth AFTER a parser error, such as "cannot find symbol" or "package does not exist". If it's a parser error (such as "`;` expected"), then Decaf will report the correct error.

In [ ]:
for scenario_id in suspected_contexts:
    scenario = ALL_SCENARIOS[scenario_id]
    decaf_output = DECAF_RESPONSES[scenario_id]
    print(scenario_id)
    print("  ", scenario["unit"].pems)
    print("  ", decaf_output)

After manually reviewing the above errors, here are the ones that I think are definitely skipable:

In [ ]:
should_skip = should_skip.union({
    ('/data/mini/srcml-2016-03/project-6635685/src-30958703.xml', '1159101311'),
    ('/data/mini/srcml-2018-06/project-12853713/src-61909914.xml', '2493733098'),
    ('/data/mini/srcml-2020-06/project-20162892/src-99474416.xml', '4532849754'),
    ('/data/mini/srcml-2019-09/project-17095277/src-83478876.xml', '3614846622'),
})

should_skip

## Creating `cleaned_df`

In [ ]:
def remove_decaf_pems(df):
    new_df = df\
        .set_index(BY_SCENARIO)\
        .drop(index=[(srcml_path, version, "decaf") for srcml_path, version in should_skip])\
        .reset_index()
    assert len(should_skip) > 0
    assert len(new_df) < len(df)
    return new_df

cleaned_df = full_df\
    .pipe(remove_decaf_pems)

cleaned_df.sample(5)

Now there should be a few fewer Decaf ratings:

In [ ]:
cleaned_df.value_counts("variant")

# Determining questions answered by at least 2 raters

Assert that all scenarios were answered by at least two raters:

In [ ]:
two_ratings = cleaned_df.groupby(["srcml_path", "version"]).filter(lambda x: len(x["rater"].unique()) >= 2)

Find messages that have only been rated by Brett. I can assign all of these to myself, and then we have a bit more data to work with:

In [ ]:
def rearrange_columns(df):
    return df[["pem_category", "srcml_path", "version"]]

def maybe_save_assignments(df):
    """
    Save new assignments to a file, if there are any scenarios that need an extra rater.
    That extra rater will be me, Eddie.
    """
    if len(df) > 0:
        df.to_csv("additional-eddie-assignments.tsv", sep="\t", index=False, header=False)
    return df

cleaned_df.groupby(["srcml_path", "version"])\
    .filter(lambda x: len(x["rater"].unique()) == 1)\
    .query('rater == "brett"')[['srcml_path', 'version']]\
    .drop_duplicates()\
    .assign(pem_category="unknown")\
    .pipe(rearrange_columns)\
    .pipe(maybe_save_assignments)

As of 2023-05-09, I assigned the remaining Brett messages to myself. The only contexts left with only one rater were assigned to me (Eddie) 🙃

In [ ]:
len(two_ratings)

# Creating `ratings`

Combine the ratings of all of raters into one, big table

In [ ]:
FIXED_COLUMNS = {'srcml_path', 'version', 'variant', 'rater', 'length'}

RESPONSE_COLUMNS = [
    column for column in full_df.columns
    if column not in FIXED_COLUMNS
]
RESPONSE_COLUMNS

## Creating the `assigned` column

In [ ]:
def add_assigned(df):
    """
    Add convenience columns that indicates whether a rater was assigned to this scenario.
    This only makes sense AFTER pivoting the DataFrame.
    """
    # A rater has given True or False if and only if they were assigned to this scenario.
    # Therefore, we can use the presence of NaNs to determine whether a rater was assigned:
    assigned = df["explanation"].notna()
    # However... we need to reconstruct the hierarchical index before we concatentate it.
    index = pd.MultiIndex.from_product([["assigned"], assigned.columns])
    assigned_df = pd.DataFrame(assigned.values, columns=index, index=assigned.index)
    # I tried to do this using DataFrame.assign, but this doesn't work for hierarchical indexes.
    return pd.concat([df, assigned_df], axis=1)

# Put all the raters side-by-side using a hierarchical index
ratings = two_ratings\
    .drop(columns=["length"])\
    .pivot(index=BY_SCENARIO, columns="rater", values=RESPONSE_COLUMNS)\
    .pipe(add_assigned)

ratings.head(8)

It's nice to see this data in an Excel spreadsheet, so OPTIONALLY export it:

In [ ]:
if globals().get("EXPORT_EXCEL", False):
    ratings.to_excel("hierarchical_responses.xlsx")

# Inter-rater agreement between individual raters

**TODO**: perhaps Scott's π is a better measure of agreement for this data:

See: https://stats.stackexchange.com/a/525640

In [ ]:
def group_two_raters(rater1, rater2):
    "Returns a DataFrame with only the rows where both raters have rated the scenario"
    return ratings[ratings["assigned"][rater1] & ratings["assigned"][rater2]]

def kappa_for_two_raters(column, rater1, rater2):
    "Returns agreement for two raters on a certain column"
    agreement = group_two_raters(rater1, rater2)
    criteria = agreement[column].astype("category")
    return cohen_kappa_score(criteria[rater1], criteria[rater2])

def kappa_for_two_raters_with_label(column, rater1, rater2):
    kappa = kappa_for_two_raters(column, rater1, rater2)
    return kappa, agreement_as_label(kappa)

In [ ]:
kappa_for_two_raters_with_label("explanation", "eddie", "prajish")

In [ ]:
kappa_for_two_raters_with_label("explanation", "eddie", "brett")

In [ ]:
kappa_for_two_raters_with_label("explanation", "prajish", "brett")

In [ ]:
kappa_for_two_raters_with_label("fix", "eddie", "prajish")

In [ ]:
kappa_for_two_raters_with_label("fix", "eddie", "brett")

In [ ]:
kappa_for_two_raters_with_label("fix", "prajish", "brett")

# Creating two synthesized raters

Synthesise two raters by null-coalescing our three raters:

In [ ]:
def create_two_raters(df, column_name):
    """
    Synthesize two raters, such that each question has at least one unique rater.
    
    This uses the following truth table to select the rater:

           Assigned?       |        Rater
     Eddie  Prajish  Brett |     1         2
    -----------------------+---------------------
      ❌      ✅       ✅   |  Brett      Prajish
      ✅      ❌       ✅   |  Eddie      Brett
      ✅      ✅       ❌   |  Eddie      Prajish
    -----------------------+---------------------
      ✅      ✅       ✅   |  Eddie      Prajish

    """
    rater1 = df[column_name]["eddie"].where(df["assigned"]["eddie"], df[column_name]["brett"])
    rater2 = df[column_name]["prajish"].where(df["assigned"]["prajish"], df[column_name]["brett"])

    return rater1, rater2

In [ ]:
NON_NULL_COLUMNS = [
    "jargon",
    "sentence_structure",
    "explanation",
    "fix",
    "additional_errors",
]


def to_correct_dtype(column, column_name):
    type_ = column_name.split("_")[-1]
    
    n_nulls = column.isnull().sum()
    if n_nulls > 0:
        assert column_name not in NON_NULL_COLUMNS, f"Column '{column_name}' has missing values"
        # Do not convert columns with nulls.
        return column

    if column_name == "perfect_fix" or type_ == "binary":
        return column.astype(bool)
    elif column_name == "jargon" or type_ == "ordinal":
        return column.astype(int)
    else:
        return column.astype("category")

# Creating the `two_raters` data frame

In [ ]:
def column_for_two_raters(df, column_name):
    rater1, rater2 = create_two_raters(df, column_name)
    index = pd.MultiIndex.from_product([[column_name], ["rater1", "rater2"]], names=[None, "rater"])
    return pd.DataFrame(
        {
            (column_name, "rater1"): to_correct_dtype(rater1, column_name),
            (column_name, "rater2"): to_correct_dtype(rater2, column_name),
        },
        columns=index, index=df.index
    )

two_raters = pd.concat([column_for_two_raters(ratings, column) for column in RESPONSE_COLUMNS], axis=1)

if globals().get("EXPORT_EXCEL", False):
    two_raters.to_excel("two_raters.xlsx")

two_raters.head(8)

In [ ]:
def two_rater_agreement(column_name, **kwargs):
    """
    Return the inter-rater reliability for a certain column for the two synthesized raters.
    Note: the column must not have any missing values.
    """
    criteria = two_raters[column_name]
    rater1 = criteria["rater1"]
    rater2 = criteria["rater2"]
    
    assert rater1.isnull().sum() == 0
    assert rater2.isnull().sum() == 0

    kappa = cohen_kappa_score(rater1, rater2, **kwargs)
    return kappa, agreement_as_label(kappa)

# Two-rater inter-rater reliability

Get the agreement for non-null categories:

In [ ]:
two_rater_agreement("sentence_structure")

For ordinal data, apply weighting to smooth the agreement.

See here for weighting: https://github.com/jmgirard/mReliability/wiki/Weighting-scheme

I think linear weighting is fair. Quadratic is waaaaay too forgiving.

In [ ]:
two_rater_agreement("sentence_structure_ordinal", weights="linear")

In [ ]:
two_rater_agreement("explanation")

In [ ]:
two_rater_agreement("fix")

Remember to report how we decided to interpret the fix categories as ordinal data:

In [ ]:
two_rater_agreement("fix_ordinal", weights="linear")

Check if Decaf data affects this: Brett and I interpretted this differently

"Researchers agreed that if Decaf found errors but did not discuss them -- turn into count"

Report count. Might not even report this for multiple errors.

IRR doesn't make much sense here:

In [ ]:
two_rater_agreement("additional_errors")

In [ ]:
two_rater_agreement("additional_errors_ordinal", weights="linear")

In [ ]:
two_rater_agreement("perfect_fix")

**TODO**: How to measure the agreement of columns with nulls?

# Results

**TODO**: descriptive statistics of jargon

**TODO**: plots of message length

**TODO**: number of additional errors reported...?

How many scenarios have been rated by each rater?

In [ ]:
full_df.groupby("rater").size()

How many variants were rated?

In [ ]:
two_raters.groupby("variant").size()

Decaf is missing 3 ratings because of the messages excluded above.

## Stack the data to obtain a long-format table

The `two_raters` data frame has a hierarchical _column_ index, but most analyses want a long-format table. To obtain this, we can stack (pivot the columns) to a column of its own.

In [ ]:
two_raters_long = two_raters.stack("rater")

two_raters_long.sample(5)

## Message length

In [ ]:
message_lengths.groupby("variant").agg({"length": ["min", "max", "mean", "std"]})

In [ ]:
sns.boxplot(data=message_lengths,
            x="variant",
            y="length",
            order=["javac", "decaf", "gpt-4-error-only", "gpt-4-with-context"])

## Jargon

In [ ]:
two_raters_long.groupby("variant").agg({"jargon": ["min", "max", "mean", "std"]})

In [ ]:
sns.displot(data=cleaned_df,
                x="jargon",
                hue="variant",
                discrete=True,
                multiple="dodge")

## Proportions of ratings

How often do the error messages provide explanations of the error?

In [ ]:
two_raters_long.groupby("variant")["explanation"].value_counts(normalize=True) * 100

In [ ]:
two_raters_long.groupby(["variant", "rater"])["explanation"].value_counts(normalize=True) * 100

In [ ]:
two_raters_long.groupby(["variant", "rater"])["explanation_correctness"].value_counts(normalize=True) * 100

In [ ]:
two_raters_long.groupby("variant")["fix"].value_counts(normalize=True) * 100

In [ ]:
two_raters_long.groupby(["variant", "rater"])["fix"].value_counts(normalize=True) * 100

In [ ]:
two_raters_long.groupby("variant")["fix_correctness"].value_counts(normalize=True) * 100

In [ ]:
two_raters_long.groupby(["variant", "rater"])["fix_correctness"].value_counts(normalize=True) * 100

In [ ]:
two_raters_long\
    .groupby("variant")["perfect_fix"]\
    .value_counts(normalize=True)\
    .sort_index() * 100

In [ ]:
two_raters_long\
    .groupby(["variant", "rater"])["perfect_fix"]\
    .value_counts(normalize=True)\
    .sort_index() * 100

**TODO**: IRR of _perfect fix_

# Scratch

In [ ]:
ALL_SCENARIOS[('/data/mini/srcml-2019-06/project-16564363/src-80577273.xml', '3470043524')]["unit"].pems[0]